In [9]:
import json 
import pandas as pd

In [46]:
df = pd.read_csv("n50-cleaned.csv")

In [47]:
df.shape

(25, 18)

In [48]:
df['answers']

0     [2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, ...
1     [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 2, ...
2     [2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, ...
3     [1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, ...
4     [1, 1, 2, 1, 2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 2, ...
5     [1, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, ...
6     [1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, ...
7     [2, 1, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 1, ...
8     [2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, ...
9     [1, 2, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 2, 1, ...
10    [1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, ...
11    [1, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, ...
12    [1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, ...
13    [2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1, 2, ...
14    [2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 1, ...
15    [2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, ...
16    [1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 1, ...
17    [2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2,

In [49]:
df['drawings']

0     [['g1_l40_a', 'g1_l65_b'], ['g3_l80_a', 'g3_l4...
1     [['g5_l65_b', 'g5_l50_c'], ['g2_l40_c', 'g2_l7...
2     [['g5_l40_a', 'g5_l80_c'], ['g2_l50_c', 'g2_l8...
3     [['g1_l50_a', 'g1_l40_c'], ['g1_l55_c', 'g1_l4...
4     [['g3_l70_b', 'g3_l45_a'], ['g4_l80_c', 'g4_l7...
5     [['g5_l65_b', 'g5_l60_a'], ['g3_l80_c', 'g3_l7...
6     [['g2_l60_c', 'g2_l65_a'], ['g5_l60_c', 'g5_l8...
7     [['g3_l40_c', 'g3_l70_b'], ['g1_l55_c', 'g1_l5...
8     [['g2_l40_a', 'g2_l75_b'], ['g5_l80_a', 'g5_l7...
9     [['g3_l65_b', 'g3_l60_c'], ['g3_l50_a', 'g3_l7...
10    [['g4_l80_a', 'g4_l40_b'], ['g2_l55_c', 'g2_l7...
11    [['g1_l60_a', 'g1_l75_b'], ['g1_l65_a', 'g1_l4...
12    [['g4_l70_b', 'g4_l70_a'], ['g3_l65_b', 'g3_l4...
13    [['g1_l45_a', 'g1_l80_c'], ['g4_l65_a', 'g4_l4...
14    [['g2_l40_b', 'g2_l70_c'], ['g1_l80_c', 'g1_l4...
15    [['g4_l75_c', 'g4_l40_a'], ['g1_l45_b', 'g1_l7...
16    [['g2_l45_b', 'g2_l80_a'], ['g4_l80_c', 'g4_l5...
17    [['g3_l75_b', 'g3_l75_a'], ['g5_l65_a', 'g

In [50]:
#Let's look at one for now. 
import re
responses = json.loads(df.iloc[0]['answers'])
drawings = json.loads(df.iloc[0]['drawings'].replace("\'", "\""))


def extract_metric(drawing_name):
    match = re.search(r'l(\d+)', drawing_name)
    return int(match.group(1)) if match else None

data = []
for row in range(df.shape[0]):
    responses = json.loads(df.iloc[row]['answers'])
    drawings = json.loads(df.iloc[row]['drawings'].replace("\'", "\""))
    for i, (pair, response) in enumerate(zip(drawings, responses)):
        metric_1 = extract_metric(pair[0])
        metric_2 = extract_metric(pair[1])
        metric_diff = metric_2 - metric_1
        
        preferred_higher_metric = 1 if (response == 2 and metric_diff > 0) or (response == 1 and metric_diff < 0) else 0
        
        data.append({'metric_diff': abs(metric_diff), 'preferred_higher_metric': preferred_higher_metric})

newdf = pd.DataFrame(data)

## Hypothesis 1: The stress of a drawing effects participant preference
Treat each response as independent trial. 
Define 0 to be a response with lower inverse stress
Define 1 to be a response with higher inverse stress
Determine if responses differ from a uniform random distribution (binomial test).
Similar analyis to <a href="https://arxiv.org/abs/2008.04869">Turing test for Graph Drawing</a> 

In [51]:
# #Accuracy column has 0 or 1 precomputed
# responses = list()
# for participant in df['accuracy'].to_list():
#     responses.extend(json.loads(participant))
# responses[:4]

In [52]:
responses = newdf['preferred_higher_metric'].to_numpy()

In [53]:
from scipy.stats import binomtest

pvalue = binomtest(
    sum(responses),
    len(responses),
    p=0.5
)

if pvalue.pvalue < 0.05: print("Responses are significantly different from a random choice")

Responses are significantly different from a random choice


In [54]:
sum(responses) / len(responses)

0.696

Let's instead treat each participant at a different factor level

In [62]:
data = dict()
for row in range(df.shape[0]):
    responses = json.loads(df.iloc[row]['answers'])
    drawings = json.loads(df.iloc[row]['drawings'].replace("\'", "\""))
    
    local_data = list()
    for i, (pair, response) in enumerate(zip(drawings, responses)):
        metric_1 = extract_metric(pair[0])
        metric_2 = extract_metric(pair[1])
        metric_diff = metric_2 - metric_1
        
        preferred_higher_metric = 1 if (response == 2 and metric_diff > 0) or (response == 1 and metric_diff < 0) else 0
        
        local_data.append(preferred_higher_metric)
    
    data[f'p{row}'] =  local_data


newdf = pd.DataFrame(data)

In [64]:
newdf.head()

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p15,p16,p17,p18,p19,p20,p21,p22,p23,p24
0,1,1,1,1,1,1,0,1,1,1,...,0,0,0,0,1,1,1,1,1,1
1,1,1,1,1,1,0,1,0,1,1,...,0,1,0,0,0,1,1,0,0,1
2,1,0,1,0,0,1,1,1,0,0,...,0,0,1,0,0,1,1,0,1,1
3,1,1,1,1,1,1,1,1,0,1,...,1,1,0,1,1,1,1,1,1,1
4,1,1,1,1,0,1,0,1,1,1,...,0,1,1,0,1,1,0,1,1,1


In [ ]:
from scipy.stats import friedmanchisquare
result = friedmanchisquare(*[newdf[col] for col in newdf.columns])

if result.pvalue < 0.05 / ((25 * 24) / 2):
    print("Significant difference")

FriedmanchisquareResult(statistic=200.22043948378095, pvalue=9.478837764357417e-30)